In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification 
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("..")
from src.models import EModel, AASIST, Wav2Vec2Facebook
from src.datamodules import AASIST2DataModule, AASISTCenterLossDataset, Wav2Vec2Dataset

In [3]:
# model_name_or_path = "facebook/wav2vec2-base"
# feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
# sampling_rate = feature_extractor.sampling_rate

path = "/home/work/joono/joono/joono/DV_DV.Deep/vunr6vff/checkpoints/best-checkpoint.ckpt"
# model = Wav2Vec2Facebook.load_from_checkpoint(path, args={})
model = Wav2Vec2Facebook.load_from_checkpoint(path, args={})

/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizat

======== Trainable Lora layers ===========
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([

In [4]:
# Collate 함수 정의
def collate_fn(batch):
    # signals = zip(*batch)
    signals = batch
    max_length = max([signal.size(0) for signal in signals])
    padded_signals = torch.zeros(len(signals), max_length)
    for i, signal in enumerate(signals):
        padded_signals[i, :signal.size(0)] = signal
    # labels = torch.tensor(labels)
    return padded_signals

In [8]:
test_df = pd.read_csv('../dataset/test.csv', index_col=None)
test_df['path'] = '../dataset/' + test_df['path'].str[1:]
# test_dataset = TestDataset(test_df)
# test_dataset = AASISTCenterLossDataset(test_df, train_mode=False)
test_dataset = Wav2Vec2Dataset(df=test_df, model_name="facebook/wav2vec2-base", train_mode=False)
# test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32, collate_fn=collate_fn)

/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [11]:
def inference(model, test_loader, device):
    model.to(device)
    model = model.eval()
    categories = []
    with torch.no_grad():
        # for inputs, labels in tqdm(test_loader):
        for inputs in tqdm(test_loader):
            
            inputs = inputs.to(device)
            
            category = model(inputs)

            categories+= category.cpu().detach().numpy().tolist()
            
    return  categories

In [13]:
categories = inference(model=model, test_loader=test_loader, device='cuda:1')
# preds = model.inference(test_loader=test_loader)

100%|██████████| 1563/1563 [00:49<00:00, 31.79it/s]


In [19]:
categories_np = np.array(categories)
categories_class = np.argmax(categories_np, axis=1)
print(len(categories_class))

categories_softmax = np.exp(categories_np) / np.sum(np.exp(categories_np), axis=1, keepdims=True)
print(len(categories_softmax))

category_probs = np.zeros((len(categories_softmax), 2))
category_count = [0, 0, 0, 0]
for row in range(len(category_probs)):
    cat_max_idx = np.argmax(categories_softmax[row])
    category_count[cat_max_idx] += 1
    prob = categories_softmax[row, cat_max_idx]
    
    print(f"cat_max: {cat_max_idx}, prob: {prob}")
    
    if      cat_max_idx == 0: category_probs[row] = np.array([1 - prob, 1 - prob])
    elif    cat_max_idx == 1: category_probs[row] = np.array([prob, 1 - prob])
    elif    cat_max_idx == 2: category_probs[row] = np.array([1 - prob, prob])
    elif    cat_max_idx == 3: category_probs[row] = np.array([prob, prob])
category_count

50000
50000
cat_max: 2, prob: 0.9996508127391492
cat_max: 2, prob: 0.9992741876420601
cat_max: 2, prob: 0.9965071120691172
cat_max: 3, prob: 0.9954244090217588
cat_max: 2, prob: 0.9972178889676939
cat_max: 3, prob: 0.6783709589486835
cat_max: 2, prob: 0.9277272574986409
cat_max: 2, prob: 0.9493770880128766
cat_max: 2, prob: 0.9983615212431949
cat_max: 3, prob: 0.5315134449100303
cat_max: 2, prob: 0.9998225273820494
cat_max: 2, prob: 0.9951716651633948
cat_max: 3, prob: 0.751377910565604
cat_max: 3, prob: 0.9967111566065121
cat_max: 2, prob: 0.9997756175497421
cat_max: 3, prob: 0.8521753582135914
cat_max: 3, prob: 0.9991294611299042
cat_max: 2, prob: 0.9950794005661079
cat_max: 2, prob: 0.8442312315241476
cat_max: 2, prob: 0.9997570781825061
cat_max: 2, prob: 0.9981066609111664
cat_max: 2, prob: 0.5115817553933477
cat_max: 2, prob: 0.9674057505086231
cat_max: 2, prob: 0.9971991643218964
cat_max: 3, prob: 0.6938829582454874
cat_max: 2, prob: 0.999886944452797
cat_max: 2, prob: 0.81093221

[0, 3124, 38510, 8366]

In [16]:
submit = pd.read_csv('/home/work/joono/joono/dataset/sample_submission.csv')

max_thres = 0.99999
min_thres = 0.00001

fake, real = 0, 1

for i in tqdm(range(len(categories_np))):
    if      category_probs[i][fake] > max_thres  : submit.iloc[i, 1] = 1
    elif    category_probs[i][fake] < min_thres  : submit.iloc[i, 1] = 0 
    else                                         : submit.iloc[i, 1] = category_probs[i][0]
    if      category_probs[i][real] > max_thres  : submit.iloc[i, 2] = 1
    elif    category_probs[i][real] < min_thres  : submit.iloc[i, 2] = 0 
    else                                         : submit.iloc[i, 2] = category_probs[i][1]

  0%|          | 0/50000 [00:00<?, ?it/s]

/tmp/ipykernel_4045649/2167967702.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.00034918726085075846' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else                                         : submit.iloc[i, 1] = category_probs[i][0]
/tmp/ipykernel_4045649/2167967702.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9996508127391492' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else                                         : submit.iloc[i, 2] = category_probs[i][1]
100%|██████████| 50000/50000 [00:08<00:00, 6062.37it/s]


In [20]:
submit.to_csv('wav2vec2-base-ce-test.csv', index=False, encoding='utf-8')

In [ ]:
import os
import pathlib
from glob import glob

In [ ]:

ids, paths = [], []
for ul_path in glob("/home/work/joono/joono/dataset/unlabeled_data/*"):
    basename = os.path.basename(ul_path)
    id = basename.split(".")[0]
    path = os.path.join(".", "unlabeled_data", basename)
    
    ids.append(id)
    paths.append(path)

In [ ]:
df = pd.DataFrame({
    "id": ids,
    "path": paths
})

In [ ]:
df.to_csv("unlabled_data.csv", index=False)

In [ ]:
paths[763]